mTSP

Multi-vehicle Traveling Sales Man
---

# Formulation

The Traveling Salesman Problem for a **complete graph** can be stated as:

$min_x$
$$ \sum_{i,j \in N, i < j} c_{ij}x_{ij} $$
$s.t$
$$\tag{1} \sum_{i \in N, i \neq j} x_{ij} = 1, \forall j \in N$$
$$\tag{2} \sum_{j \in N, i \neq j} x_{ij} = 1, \forall i \in N$$
$$\tag{3} u_i - u_j + |N|x_{ij} \le |N|-1 , \forall (i,j) \in
N\setminus\lbrace{1}\rbrace, i \neq j$$

$$\tag{4} \sum_{i \in N\setminus\lbrace{1}\rbrace} x_{1j} = m$$
$$\tag{5} \sum_{j \in N\setminus\lbrace{1}\rbrace} x_{i1} = m$$

# Solve with Cplex

In [2]:
%%file "../pkg/mtsp_cplex.py"

import numpy as np              # mathematic tools library
import networkx as nx           # network representation library
from pkg.cplex_solve import cplex_solve
import cplex

def mtsp_cplex(N,m
            c,
            relaxation=False,path=None):

    #####################################################################
    # Decision variables
    
    def X(i,j):
        return "X_" + str(i) + "_" + str(j)
    
    def U(i):
        return "U_" + str(i)
    
    N = range(N);
    #####################################################################
    # Objective function
    
    obj1 = [c[i][j] for i in N for j in N]
    obj2 = [0 for i in N]
    
    ## variables name
    Xs = [X(i,j) for i in N for j in N]
    Us = [U(i) for i in N]

    ## Objective function sum aggregation
    obj = obj1 + obj2
    colnames = Xs + Us
    if relaxation:
        types    = "C" * (len(Xs)+len(Us)) #Integrality constraint
    else:
        types    = "I" * len(Xs) + "C" * len(Us) #Integrality constraint

    #####################################################################
    # Constraints
    
    c1 = [
            [[X(i,j) for i in N if i != j], [1 for i in N if i != j]]
        for j in N]
    
    c2 = [
            [[X(i,j) for j in N if i != j], [1 for j in N if i != j]]
        for i in N]
    
    subtours = [(i,j) for i in N for j in N if i != j and j > 0 and i > 0]
    
    c3 = [
            [[U(i),U(j),X(i,j)], [1,-1,len(N)-1]]
        for i,j in subtours]
    
    c4 = [
            [[X(i,0) for i in N if i > 0], [1 for i in N if i > 0 ]]
        ]
    
    c4 = [
            [[X(0,j) for i in N if j > 0], [1 for j in N if i > 0 ]]
        ]
    
    s1 = "E" * len(N)
    s2 = "E" * len(N)
    s3 = "L" * len(subtours)
    s4 = "E"
    s5 = "E"

    r1 = [1 for i in N]
    r2 = [1 for i in N]
    r3 = [len(N)-2 for i in range(len(subtours))]
    r4 = m

    rows = c1+c2+c3
    senses = s1+s2+s3
    rhs =  r1+r2+r3
    
    #####################################################################
    # Bounds
    ub = [1 for i in range(len(Xs))] + [cplex.infinity for i in range(len(Us))]
    lb = [0 for i in range((len(Xs)+len(Us)))]
    
    #####################################################################
    # Solving
    prob = cplex_solve(obj,ub,lb,colnames,types, rows, senses, rhs, minimize=True, path=path)

    #####################################################################
    # Extract solution
    N = len(N)
    
    solution = prob.solution.get_values()
    X = np.reshape(solution[0:N*N],(N,N))

    return prob, X

Overwriting ../pkg/mtsp_cplex.py
